In [1]:
from wordcloud import WordCloud, STOPWORDS 
import numpy as np
import matplotlib.pyplot as plt 
import mysql.connector
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
cnx = mysql.connector.connect(user='usr_tronexotrs', password='usr$otrs#56aN7Rt*mp1',
                              host='pantermysql.cwqmr5tsjjh9.us-west-2.rds.amazonaws.com',
                              database='tronex.otrs')

In [3]:
query = ('SELECT T.title, S.* FROM `tronex.otrs`.itv_sabana_tickets AS S INNER JOIN `tronex.otrs`.ticket AS T \
         ON S.ticket_id = T.id WHERE cola = "SERVICIOS IT" AND \
         correo_cliente NOT IN ("soporte@sasaconsultoria.com", "telefonia@tronex.com"\
         , "serviciosit@tronex.com") AND T.title not like "%has low free space" AND YEAR(S.fecha_creacion)=2021')
df = pd.read_sql(query, con=cnx)

In [4]:
df['cantidad_servicio'] = 1
df2 = df.pivot_table(index='fecha_creacion', \
                     columns='servicio', \
                     aggfunc='sum', \
                     values='cantidad_servicio', \
                     fill_value=0)
df2.drop(columns=['OFIMATICA::EQUIPOS DE COMPUTO', 'OFIMATICA::APLICACIONES DE ESCRITORIO'], inplace=True)
def encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
df2 = df2.applymap(encode)

In [5]:
df_freqserv = apriori(df2, min_support=0.3, use_colnames=True)
df3 = association_rules(df_freqserv, min_threshold=1, metric='lift')

In [6]:
df3.loc[(df3.confidence>0.5)].sort_values('confidence', ascending=False).head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
50,"(OFIMATICA::PERIFERICO, IMPRESION ::IMPRESION)",(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.357143,0.628571,0.300000,0.840000,1.336364,0.075510,2.321429
34,(OFIMATICA::PERIFERICO),(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.485714,0.628571,0.400000,0.823529,1.310160,0.094694,2.104762
44,"(OFIMATICA::PERIFERICO, ERP::ADMINISTRACION)",(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.371429,0.628571,0.300000,0.807692,1.284965,0.066531,1.931429
3,(ACCESIBILIDAD::DIRECTORIO ACTIVO),(IMPRESION ::IMPRESION),0.442857,0.614286,0.357143,0.806452,1.312828,0.085102,1.992857
36,(TELEFONIA::TELEFONIA IP),(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.428571,0.628571,0.342857,0.800000,1.272727,0.073469,1.857143
7,(AUTOMATIZACION FUERZA DE VENTAS::ADMINISTRACION),(ERP::ADMINISTRACION),0.471429,0.600000,0.371429,0.787879,1.313131,0.088571,1.885714
13,(AUTOMATIZACION FUERZA DE VENTAS::ADMINISTRACION),(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.471429,0.628571,0.371429,0.787879,1.253444,0.075102,1.751020
39,"(IMPRESION ::IMPRESION, ERP::ADMINISTRACION)",(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.442857,0.628571,0.342857,0.774194,1.231672,0.064490,1.644898
26,(IMPRESION ::IMPRESION),(MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS),0.614286,0.628571,0.471429,0.767442,1.220930,0.085306,1.597143
30,(TELEFONIA::TELEFONIA IP),(IMPRESION ::IMPRESION),0.428571,0.614286,0.328571,0.766667,1.248062,0.065306,1.653061


In [13]:
df.shape

(1211, 36)

In [14]:
df[df['servicio'].isin(['TELEFONIA::TELEFONIA IP', 'ERP::ADMINISTRACION'])].shape

(124, 36)

In [15]:
df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'])

In [17]:
df4 = df.set_index('fecha_creacion').groupby(['servicio', pd.Grouper(freq='D')]).count().reset_index('servicio')

In [18]:
df4.shape

(543, 35)

In [23]:
df4.index.nunique()

66

In [30]:
df4[df4['servicio'].isin(['MENSAJERIA Y COLABORACION::CORREO ELECTRONICO'])].index.nunique()

30